Step 4

In [227]:
##imports##
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import math


Step 5

In [228]:
##read data##

#step 8#
file_path_layout_0 = "/Users/zach/Documents/Education/Undergrad (UWEC)/Summer 2023/UCLA/optimization_protocol/06-07-2023-3HC-standard-protocol/Pre_label_od_layout.csv"
file_path_data_0 = "/Users/zach/Documents/Education/Undergrad (UWEC)/Summer 2023/UCLA/optimization_protocol/06-07-2023-3HC-standard-protocol/Pre-label-od.csv"

file_path_layout_1 = "/Users/zach/Documents/Education/Undergrad (UWEC)/Summer 2023/UCLA/optimization_protocol/06-07-2023-3HC-standard-protocol/post-label-od-flo-layout.csv"
file_path_data_1 = "/Users/zach/Documents/Education/Undergrad (UWEC)/Summer 2023/UCLA/optimization_protocol/06-07-2023-3HC-standard-protocol/labeled-od.csv"



##if file .txt and not .csv then include an r before the ""parentheses""
##thus it will be 
##file_path_data = r"##please input the file path or the file name##"

##copy and paste this as many times as the files you have
##if you have 4 files make sure there are 8 file paths, 4 for the data, 4 for the layout


df_plate_reader_layout_1 = pd.read_csv(file_path_layout_0)
df_plate_reader_data_1 = pd.read_csv(file_path_data_0)

df_plate_reader_layout_2 = pd.read_csv(file_path_layout_1)
df_plate_reader_data_2 = pd.read_csv(file_path_data_1)


Step 6

In [229]:
##change the size of the dictionary depending on how many files you have imported, thus if you have 4 plate reader data, 
## your dictionary size should be 4 with a list of 2 dataframes at each index
a_dict_of_all_data = {
    0: [df_plate_reader_layout_0, df_plate_reader_data_0], 
    1: [df_plate_reader_layout_1, df_plate_reader_data_1], 
                      }

Step 7

Functions

In [230]:
## Is Nan function

def isNaN(num):
    return num != num

In [75]:
##function that finds the number of groups in the layout
def unique_samples_from_plate_layout(df):
    df = df.iloc[:, 1:].reset_index(drop=True)
    samples = set()

    for i in range(df.shape[0] - 1):
        for j in range(df.shape[1] - 1):
            unique_sample = df.iloc[i, j]

            if pd.isna(unique_sample):
                continue

            samples.add(unique_sample)

    return list(samples)

In [76]:
##finding values in the data that are mapped to the same layout position
def find_values(string, layout, data_frame):
    layout = layout.iloc[:, 1:]
    data_frame = data_frame.iloc[:, 1:]
    values = []

    for ii in range(layout.shape[0]):
        for jj in range(layout.shape[1]):
            if layout.iloc[ii, jj] == string:
                values.append(data_frame.iloc[ii, jj])

    return values

In [7]:
##Function that enumerates through the groups of the layout
##Extracts the data in the corresponding plate map
##placing this information into the correct column
def developing_new_df_from_input_data(layout, data_plate_reader):
    column_titles = unique_samples_from_plate_layout(layout)
    num_rows = layout.shape[0]
    num_cols = layout.shape[1]

    df = pd.DataFrame(index=range(num_rows), columns=column_titles)

    for ii, title in enumerate(column_titles):
        cell_position = 0

        for jj in range(num_rows):
            for kk in range(num_cols):
                if layout.iloc[jj, kk] == title:
                    df.iloc[cell_position, ii] = data_plate_reader.iloc[jj, kk]
                    cell_position += 1

    return df

In [8]:
##A function to obtain the original density after a dilution
def get_OD(ratio, final_volume, cell_position):
    ratio_nums = ratio.split(':')
    multiplyer = int (int(ratio_nums[1]) / int(ratio_nums[0]) )
    a = multiplyer*final_volume*cell_position
    return a

In [9]:
##Function to calculate the original density and put it into the dataframe
def calculate_the_original_density(df, columns_names_ajusted, diluted_to, final_volume):
    for ii in range(len(columns_names_ajusted)):

        name = columns_names_ajusted[ii]

        df[str(name)] = get_OD(str(diluted_to[ii]), float(final_volume), df[str(name)])
        
    return df

In [10]:
def finding_statistically_significant_groups_update(list_of_columns, p_value, dataframe_df, scaling_factor):
    import matplotlib.pyplot as plt
    import seaborn as sns
    import itertools
    import scipy.stats as stats

    # Set up the plot
    sns.set(style='whitegrid')
    fig, axes = plt.subplots(1, 1, figsize=(12, 6), dpi=600)
    new_df = dataframe_df[list_of_columns]

    # Plot the data
    sns.boxplot(data=new_df, ax=axes)

    boxplot_height = axes.get_ylim()[1]  # Get the maximum height of the box plots

    line_offset = scaling_factor  # Adjust this value to control the spacing between the lines

    for i in range(len(list_of_columns)):
        for j in range(i+1, len(list_of_columns)):
            t_stat, p_val = stats.ttest_ind(new_df.iloc[:, i], new_df.iloc[:, j])

            if p_val <= p_value:
                x_pos = (i + j) / 2
                y_pos = boxplot_height + line_offset  # Adjust the y-coordinate to be above the previous lines

                plt.text(x_pos, y_pos, f"p={p_val:.2e}", ha='center', va='bottom', fontsize=8)

                plt.plot([i, j], [y_pos, y_pos], 'k--')

                line_offset += scaling_factor  # Increase the line offset for the next line

    return fig

In [ ]:
##Function that enumerates through the groups of the layout
##Extracts the data in the corresponding plate map
##placing this information into the correct column
def developing_new_df_from_input_data(layout, data_plate_reader):
    column_titles = unique_samples_from_plate_layout(layout)
    num_rows = layout.shape[0]
    num_cols = layout.shape[1]

    df = pd.DataFrame(index=range(num_rows), columns=column_titles)

    for ii, title in enumerate(column_titles):
        cell_position = 0

        for jj in range(num_rows):
            for kk in range(num_cols):
                if layout.iloc[jj, kk] == title:
                    df.iloc[cell_position, ii] = data_plate_reader.iloc[jj, kk]
                    cell_position += 1

    return df

In [199]:
##Function that enumerates through the groups of the layout
##Extracts the data in the corresponding plate map
##placing this information into the correct column
def developing_new_df_from_multi_input_data(a_dictionary):
    ##looping through how many files we have
    for xx in range(len(a_dictionary)):

        ##Creating the initial dataframe from the first data and layout
        if xx==0:
            #identifying the unique columns
            column_titles = unique_samples_from_plate_layout(a_dictionary[xx][0])

            ##determining the shape of this dataframe
            num_rows = a_dictionary[xx][0].shape[0]
            num_cols = a_dictionary[xx][0].shape[1]

            ##creating an empty dataframe the size determined above
            df = pd.DataFrame(index=range(num_rows), columns=column_titles)

            ##iterating through each of the column names, extracting that data from the layout and the same position in the data, and placing this into the column with the respective name
            for ii, title in enumerate(column_titles):
                cell_position = 0

                for jj in range(num_rows):
                    for kk in range(num_cols):
                        if a_dictionary[xx][0].iloc[jj, kk] == title:
                            df.iloc[cell_position, ii] = a_dictionary[xx][1].iloc[jj, kk]
                            cell_position += 1

        
        else:
            ##identifing unique column names for the subsequent data layouts 
            new_column_titles = unique_samples_from_plate_layout(a_dictionary[xx][0])

            ##creating a new dataframe containing the names of the columns that have been renamed
            new_df = pd.DataFrame(index=range(num_rows), columns=new_column_titles)
            
            ##iterating through each of the column names, extracting that data from the layout and the same position in the data, and placing this into the column with the respective name
            for ii, title in enumerate(new_column_titles):
                cell_position = 0

                for jj in range(num_rows):
                    for kk in range(num_cols):
                        if a_dictionary[xx][0].iloc[jj, kk] == title:
                            new_df.iloc[cell_position, ii] = a_dictionary[xx][1].iloc[jj, kk]
                            cell_position += 1

            ##concatenating these newly made dataframes
            df = pd.concat([df, new_df], axis=1)

    # Find the column with the maximum non-NaN values
    df_cleaned = df.dropna(how='all')

    # Find duplicate column names
    duplicate_columns = df_cleaned.columns[df_cleaned.columns.duplicated()]

    # Iterate through duplicate columns
    for column_name in duplicate_columns:
        column_values = []  # List to store values from all columns with the same name
        num_columns = df_cleaned[column_name].shape[1]  # Get the number of columns with the same name
        
        # Iterate through each column with the same name and extract values
        for jj in range(num_columns):
            column_values.append(df_cleaned[column_name].iloc[:, jj].tolist())  # Convert to list
        
        flattened_list = [item for sublist in column_values for item in sublist]
        
        # Ensure the flattened list length matches the length of the DataFrame index
        if len(flattened_list) == len(df.index):
            df_cleaned[column_name] = flattened_list
        else:
            # Insert NaN values to replace missing values of the dataframe
            cleaned_list = [value for value in flattened_list if not pd.isnull(value)]

            df1 = pd.DataFrame({column_name:cleaned_list})
            df_cleaned = df_cleaned.drop(column_name, axis=1)

            # Reindex the smaller dataframe with NaN values to match the larger dataframe's index
            df_cleaned = df_cleaned.reindex(df1.index, fill_value=np.nan)

            # Concatenate the dataframes along rows
            df_cleaned = pd.concat([df1, df_cleaned], axis=1)
            
    return df_cleaned

In [234]:
##this will be a function where columns can be combined 
##this will help when multiple people are have varying naming convenitons
def renaming_columns_of_a_dataframe(dataframe, names_that_are_the_same, names_they_should_be):
    for ii in range(len(names_that_are_the_same)):
        column_values = []  # List to store values from all columns with the same name
        num_columns = dataframe[names_that_are_the_same[ii]].shape[1]  # Get the number of columns with the same name
            
        # Iterate through each column with the same name and extract values
        for jj in range(num_columns):
            column_values.append(dataframe[names_that_are_the_same[ii]].iloc[:, jj].tolist())  # Convert to list
            
        flattened_list = [item for sublist in column_values for item in sublist]
        
        if len(flattened_list) == len(dataframe.index):
            dataframe = dataframe.drop(names_that_are_the_same[ii], axis=1)
            dataframe[names_they_should_be[ii]] = flattened_list
        elif len(flattened_list) > len(dataframe.index):
             # Insert NaN values to replace missing values of the dataframe
            cleaned_list = [value for value in flattened_list if not pd.isnull(value)]

            df1 = pd.DataFrame({names_they_should_be[ii]:cleaned_list})
            dataframe = dataframe.drop(names_that_are_the_same[ii], axis=1)
            # Reindex the smaller dataframe with NaN values to match the larger dataframe's index
            dataframe = dataframe.reindex(df1.index, fill_value=np.nan)
             # Concatenate the dataframes along rows
            dataframe = pd.concat([df1, dataframe], axis=1)

        elif len(flattened_list) < len(dataframe.index):        
            num_nan_values = len(dataframe.index) - len(flattened_list)
            extended_list = flattened_list + [np.nan] * num_nan_values
            if len(extended_list) != len(dataframe.index):
                extended_list = extended_list[:len(dataframe.index)]  # Trim the list if it's still too long
            dataframe[names_they_should_be[ii]] = extended_list
        
    return dataframe

Step 8

In [ ]:
df = developing_new_df_from_multi_input_data(a_dict_of_all_data)

Step 9

In [ ]:
print(df)
print(df.columns)

Step 10

In [239]:
similar_names = [
    ["blank (300ul)", "blank (200ul)"],
    ["1:10 (300uL) T2", "1:10 (200uL) T2", "1:10 (200uL) T1", "1:10 (300uL) T1"],
    ["1:1 (200uL) T1", '1:1 (300uL) T2', "1:1 (200uL) T2", "1:1 (300uL) T1"],
    ["1:5 (200uL) T1", "1:5 (200uL) T2", "1:5 (300uL) T1", "1:5 (300uL) T1", '1:5 (300uL) T2'],
    ['DPBS (100uL)','DPBS (300uL)','DPBS (200uL)'],
    ['3HC T1 (100uL)','3HC T1 (300uL)','3HC T1 (200uL)'],
    ['Control T2 (300uL)','Control T2 (200uL)','Control T1 (300uL)','Control T2 (100uL)','Control T1 (100uL)','Control T1 (200uL)']
]
wanted_names = [
    "blank", 
    "1:10",
    "1:1",
    "1:5",
    'DPBS', 
    '3HC',
    'Control'
]

Step 11

In [240]:
df = renaming_columns_of_a_dataframe(df, similar_names, wanted_names)

[0.001, 0.003, 0.0, 0.004, nan, -0.002, -0.002, -0.003, -0.003, nan]
[0.081, 0.085, nan, nan, nan, nan, nan, nan, 0.048, 0.039, nan, nan, nan, nan, nan, nan, 0.051, 0.022, nan, nan, nan, nan, nan, nan, 0.075, 0.08, nan, nan, nan, nan, nan, nan]
[0.398, 0.358, nan, nan, nan, nan, nan, nan, 0.654, 0.686, nan, nan, nan, nan, nan, nan, 0.46, 0.408, nan, nan, nan, nan, nan, nan, 0.623, 0.677, nan, nan, nan, nan, nan, nan]
[0.098, 0.059, nan, nan, nan, nan, nan, nan, 0.105, 0.088, nan, nan, nan, nan, nan, nan, 0.15, 0.154, nan, nan, nan, nan, nan, nan, 0.15, 0.154, nan, nan, nan, nan, nan, nan, 0.159, 0.157, nan, nan, nan, nan, nan, nan]
[0.03, 0.029, 0.034, nan, nan, nan, nan, nan, nan, nan, -0.021, -0.2, -0.022, nan, nan, nan, nan, nan, nan, nan, -0.01, -0.01, -0.009, nan, nan, nan, nan, nan, nan, nan]
[1.654, nan, nan, nan, nan, nan, nan, nan, nan, 2.087, 1.757, 2.002, nan, nan, nan, nan, nan, nan, 1.953, 1.864, 1.982, nan, 2.02, nan, nan, nan, nan]
[1.407, 1.396, 1.492, nan, nan, nan, na

,Control,3HC,DPBS,1:5,1:1,1:10,blank
0,1.407,1.654,0.030,0.098,0.398,0.081,0.001
1,1.396,2.087,0.029,0.059,0.358,0.085,0.003
2,1.492,1.757,0.034,0.105,0.654,0.048,0.000
3,1.594,2.002,-0.021,0.088,0.686,0.039,0.004
4,1.593,1.953,-0.200,0.150,0.460,0.051,-0.002
5,1.404,1.864,-0.022,0.154,0.408,0.022,-0.002
6,0.974,1.982,-0.010,0.150,0.623,0.075,-0.003
7,0.930,2.020,-0.010,0.154,0.677,0.080,-0.003
8,0.971,NaN,NaN,NaN,NaN,NaN,NaN
9,1.974,NaN,NaN,NaN,NaN,NaN,NaN


Step 12:

In [204]:
columns_need_to_be_adjusted = [
    '##change name to group name##', 
    '##change name to group name##'
    ]
diluted_from_original_density = [
    '##change to correct dilution##', 
    '##change to correct dilution##'
    ]
final_volume_in_ml = ##change to correct final volume amount##

In [ ]:
df = calculate_the_original_density(df,columns_need_to_be_adjusted, diluted_from_original_density, final_volume_in_ml)

Step 13

In [ ]:
##Graphing the data

#set up the style, personally I am a fan of 'whitegrid'
sns.set(style='##choose your style##')

#setting up the figure, axes, and figure size
fig, axes = plt.subplots(1, 3, figsize=(12, 6), dpi = 600)

#Choosing the graphs and the x-axis groupins
graph1 = ['##group 1##', '##group 2##', '##group 3##', '##group 4##',
           '##group 5##', '##group 6##', '##group 7##', '##group 8##']
graph2 = ['##group 1##', '##group 2##', '##group 3##', '##group 4##',
           '##group 5##', '##group 6##', '##group 7##', '##group 8##']
graph3 = ['##group 1##', '##group 2##', '##group 3##', '##group 4##',
           '##group 5##', '##group 6##', '##group 7##', '##group 8##']

#plotting the data
sns.barplot(data=df[graph1], ax=axes[0])
sns.lineplot(data=df[graph2], ax=axes[1])
sns.boxplot(data=df[graph3], ax=axes[2])

#choosing the labels and titles
axes[0].set_xlabel('##set x label##')
axes[0].set_ylabel('##set y label##')
axes[0].set_title('## set title##')


axes[1].set_xlabel('##set x label##')
axes[1].set_ylabel('##set y label##')
axes[1].set_title('## set title##')


axes[2].set_xlabel('##set x label##')
axes[2].set_ylabel('##set y label##')
axes[2].set_title('## set title##')

#figure supertitle
fig.suptitle('##set figure super title##', fontsize=16)

#ploting the graphs
plt.tight_layout()
 
##saving the data and graphs

##plt.savefig('##please put the name that you want##.png' )

#showing us the data
plt.show()


Step 14

In [ ]:
fig, axes = finding_statistically_significant_groups_update(##list of groups##, ##pvalue##, df, scaling_factor)
axes.set_ylabel(##choose you y label)
axes.set_title(##choose you title)

##plt.savefig('##save file to name you want##.png')